# LSTM 

In [33]:
from pathlib import Path
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Add src folder to Python path
sys.path.append(str(Path("../src").resolve()))

from data_generation import generate_data
from lstm import WindowedTimeSeries, train_model, predict_one_step, predict_autoregressive

In [ ]:
series = generate_data(10000, noise_level=0.05)
train_series = series[:, :8000]
val_series = series[:, 8000:]

In [16]:
train_dataset = WindowedTimeSeries(train_series, 10)
val_dataset = WindowedTimeSeries(val_series, 10)

In [19]:
model = train_model(
  train_dataset=train_dataset,
  val_dataset=val_dataset,
  hidden_size=100,
  num_layers=1,
  batch_size=64,
  learning_rate=0.01,
  num_epochs=100)

Epoch [10/100], train_loss: 1.0068, val_loss: 0.9772
Epoch [20/100], train_loss: 0.9311, val_loss: 0.9055
Epoch [30/100], train_loss: 0.9082, val_loss: 0.8804
Epoch [40/100], train_loss: 0.8981, val_loss: 0.8811
Epoch [50/100], train_loss: 0.8927, val_loss: 0.8709
Epoch [60/100], train_loss: 0.8834, val_loss: 0.9074
Epoch [70/100], train_loss: 0.8965, val_loss: 0.8965
Epoch [80/100], train_loss: 0.8689, val_loss: 0.8784
Epoch [90/100], train_loss: 0.8759, val_loss: 0.8243
Epoch [100/100], train_loss: 0.8702, val_loss: 0.9042


In [38]:
test_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [21]:
X_batch, y_batch = next(iter(test_loader))

print("X_batch shape:", X_batch.shape)
print("y_batch shape:", y_batch.shape)

X_batch shape: torch.Size([64, 10, 3])
y_batch shape: torch.Size([64, 3])


In [8]:
all_preds = []


for X_batch, _ in test_loader:   # Ignore targets
    preds = predict_one_step(model, X_batch)
    all_preds.append(preds)

all_preds = torch.cat(all_preds)

In [9]:
# Take first window from batch
single_window = X_batch[0]  # shape: (window_size, input_size)

steps = 20  # number of steps to predict
pred_single = predict_autoregressive(model, single_window, steps=steps)

print("Single window prediction shape:", pred_single.shape)
# Expect: (1, steps, output_size) or (steps, output_size) depending on implementation



Single window prediction shape: torch.Size([1, 3])


In [11]:
pred_single

tensor([[-15.4183, -15.2146,  26.8224]])

In [22]:
steps = 10  # number of steps to predict ahead

# Make sure your model is on the same device
device = next(model.parameters()).device
X_batch = X_batch.to(device)

# Predict
pred_batch = predict_autoregressive(model, X_batch, steps=steps, device=device)

print("Batch prediction shape:", pred_batch.shape)
# Expect: (batch_size, output_size) if returning only last prediction

Batch prediction shape: torch.Size([64, 3])


In [54]:
all_preds = []
all_target = []

for X_batch, Y_batch in test_loader:   # Ignore targets
    preds = predict_autoregressive(model, X_batch, steps=10)
    all_preds.append(preds)
    all_target.append(Y_batch)

all_preds = torch.cat(all_preds)
all_target = torch.cat(all_target)
nn.MSELoss()(all_preds[10:,:], all_target[10:,:])


tensor(157.3008)

In [51]:
all_preds[10:, :].shape

torch.Size([1980, 3])

In [63]:
steps = [1,2,3,4,5,10,15,20]

for step in range(1,21):
  all_preds = []
  all_target = []
  for X_batch, Y_batch in test_loader:
    preds = predict_autoregressive(model, X_batch, steps=step)
    all_preds.append(preds)
    all_target.append(Y_batch)


  all_preds = torch.cat(all_preds)
  all_target = torch.cat(all_target)
  
  # Corrects allignment due to predicting multiple steps
  if step != 1:
    all_preds = all_preds[:-(step-1), :]
    all_target = all_target[(step-1):, :]

  mse = nn.MSELoss()(all_preds, all_target)
  print(f"{step} Steps, MSE: {mse}")


1 Steps: MSE: 0.9041944742202759
2 Steps: MSE: 1.0372573137283325
3 Steps: MSE: 1.2958298921585083
4 Steps: MSE: 1.7358133792877197
5 Steps: MSE: 2.5600857734680176
6 Steps: MSE: 4.016411304473877
7 Steps: MSE: 6.517041206359863
8 Steps: MSE: 10.282163619995117
9 Steps: MSE: 15.778282165527344
10 Steps: MSE: 23.138715744018555
11 Steps: MSE: 31.948261260986328
12 Steps: MSE: 40.09413528442383
13 Steps: MSE: 44.97219467163086
14 Steps: MSE: 47.571590423583984
15 Steps: MSE: 50.01979446411133
16 Steps: MSE: 53.04625701904297
17 Steps: MSE: 55.42801284790039
18 Steps: MSE: 57.69843292236328
19 Steps: MSE: 61.08251953125
20 Steps: MSE: 64.44770812988281
